# D121 - n+1, n+2, n+3 timespans

In [1]:
import hyp3_sdk as sdk
import asf_search as asf
import geopandas as gpd
import pandas as pd
import os
import copy

In [2]:
hyp3 = sdk.HyP3() # 'https://hyp3-api.asf.alaska.edu'
print('User:{user_id} Quota:{quota}'.format(**hyp3.my_info()))

User:scottyhq Quota:{'max_jobs_per_month': 3000, 'remaining': 2983}


In [3]:
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          86.870901, 
          27.904352
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)

In [4]:
results = asf.search(
                    platform=[asf.PLATFORM.SENTINEL1],
                    processingLevel=asf.BURST,
                    beamMode=asf.BEAMMODE.IW,
                    #relativeOrbit=relorb,
                    polarization='VV',
                    #flightDirection='ASCENDING',
                    intersectsWith=str(gfa.geometry.values[0]),
                    #start='2018-01-01',
                    #end='2023-09-01',
                    #season=[180,270], #first,last day of year June~30*6=180
                    )
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
print(len(gf))
# Useful to fine burstIDs of interest

34


In [5]:
gf['burstId'] = gf.burst.str['fullBurstID']
gf.groupby('burstId').sceneName.count()

burstId
012_023790_IW1    18
121_258661_IW2    16
Name: sceneName, dtype: int64

In [6]:
# Pick a single burst ID to work with
burst = '121_258661_IW2'

In [7]:
# Look at timeline of acquisition
gfb = gf[gf.burstId == burst]
gfb['datetime'] = pd.to_datetime(gfb.startTime)

/Users/scott/miniforge3/envs/hyp3/lib/python3.12/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
gfb = gfb.sort_values('datetime') # chronological ascending

In [9]:
gfb.datetime.describe()

count                                     16
mean     2023-10-04 12:11:50.763450368+00:00
min         2023-06-17 00:11:47.170650+00:00
25%      2023-08-25 00:11:51.198982400+00:00
50%      2023-10-09 00:11:52.520427008+00:00
75%      2023-11-23 00:11:51.802771968+00:00
max         2024-01-07 00:11:49.624996+00:00
Name: datetime, dtype: object

In [10]:
gfb.datetime.diff().describe()

count                            15
mean     13 days 14:24:00.163623066
std       6 days 04:43:21.685988332
min         11 days 23:59:59.099228
25%      11 days 23:59:59.584068500
50%         12 days 00:00:00.035523
75%      12 days 00:00:00.494880500
max         36 days 00:00:01.856095
Name: datetime, dtype: object

In [11]:
subset = gfb.set_index('datetime')
#subset = subset[subset.index.month == 7] 
# Lots of selection opytion - random, first, last,e tc
#subset = subset.groupby([subset.index.year, subset.index.month]).first()
len(subset.fileID.to_list())

16

In [12]:
subset['date'] = pd.to_datetime(subset.startTime) #.dt.strftime('%Y%m%d')

In [13]:
job_definition = {
  "name": None,
  "job_type": "INSAR_ISCE_BURST",
  "job_parameters": {
      "granules": [None,None],
      "apply_water_mask": False, #default=False not enabled?
      "looks": "5x1", # '20x4', '10x2', '5x1' default=20x4
  },
}
job_definition

{'name': None,
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': [None, None],
  'apply_water_mask': False,
  'looks': '5x1'}}

In [14]:
# create interferogram job definitions
jobs = [] # List of job dictionaries

for n in [1,2,3]:
    for i in range(len(subset)-n):
        jobdef = copy.deepcopy(job_definition) #Avoid overwriting in-memory!
        
        ref = subset.iloc[i]
        sec = subset.iloc[i+n]
        jobname = f"{burst}_{ref.date.strftime('%Y%m%d')}_{sec.date.strftime('%Y%m%d')}_5rlks"

        jobdef['name'] = jobname
        jobdef['job_parameters']['granules'] = [ref.fileID, sec.fileID]
        jobs.append(jobdef)

In [15]:
print(len(jobs))
# NOTE: each job should complete in ~20minutes
jobs[0]

42


{'name': '121_258661_IW2_20230617_20230629_5rlks',
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': ['S1_258661_IW2_20230617T001146_VV_EB5D-BURST',
   'S1_258661_IW2_20230629T001146_VV_8416-BURST'],
  'apply_water_mask': False,
  'looks': '5x1'}}

In [16]:
jobs[1]

{'name': '121_258661_IW2_20230629_20230711_5rlks',
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': ['S1_258661_IW2_20230629T001146_VV_8416-BURST',
   'S1_258661_IW2_20230711T001147_VV_6F6C-BURST'],
  'apply_water_mask': False,
  'looks': '5x1'}}

In [17]:
# For any given reference date we should have 3 jobs:
[job for job in jobs if job['job_parameters']['granules'][0].startswith('S1_258661_IW2_20230629')]

[{'name': '121_258661_IW2_20230629_20230711_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_258661_IW2_20230629T001146_VV_8416-BURST',
    'S1_258661_IW2_20230711T001147_VV_6F6C-BURST'],
   'apply_water_mask': False,
   'looks': '5x1'}},
 {'name': '121_258661_IW2_20230629_20230816_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_258661_IW2_20230629T001146_VV_8416-BURST',
    'S1_258661_IW2_20230816T001149_VV_E142-BURST'],
   'apply_water_mask': False,
   'looks': '5x1'}},
 {'name': '121_258661_IW2_20230629_20230828_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_258661_IW2_20230629T001146_VV_8416-BURST',
    'S1_258661_IW2_20230828T001150_VV_A75C-BURST'],
   'apply_water_mask': False,
   'looks': '5x1'}}]

## Submit!


In [18]:
jobs = hyp3.submit_prepared_jobs(jobs)

In [19]:
# monitor jobs
batch = hyp3.find_jobs(job_type='INSAR_ISCE_BURST')

In [20]:
# NOTE: up to 85 running simultaneously
df = pd.DataFrame([x.to_dict() for x in batch])
df = df[df.name.str.startswith(burst)]
df.status_code.value_counts()

status_code
PENDING    42
Name: count, dtype: int64